# 读取CSV文件

In [1]:
import pandas as pd
import jieba
import numpy as np

In [2]:
content=pd.read_csv(r'/Users/yang/Downloads/datasource-master/sqlResult_1558435.csv',encoding='gb18030')
content.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [3]:
content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89611 entries, 0 to 89610
Data columns (total 7 columns):
id         89611 non-null int64
author     79396 non-null object
source     89609 non-null object
content    87054 non-null object
feature    89611 non-null object
title      89577 non-null object
url        87144 non-null object
dtypes: int64(1), object(6)
memory usage: 4.8+ MB


# 数据预处理

In [4]:
corpus=[' '.join(list(jieba.cut(str(content.iloc[i].content)))) for i in range(60000)]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/kl/7_vv7v_15wnf5q65wh8tl5jh0000gn/T/jieba.cache
Loading model cost 2.214 seconds.
Prefix dict has been built succesfully.


In [5]:
source=[content.iloc[i].source for i in range(60000)]

In [6]:
target=[1 if i=='新华社' or i=='新华网' else 0 for i in source]

In [7]:
y=np.array(target)

In [8]:
y

array([0, 0, 0, ..., 1, 1, 1])

#### sklearn.feature_extraction.text  之 CountVectorizer用法 示例

from sklearn.feature_extraction.text import CountVectorizer
corpus1 = [
...     'This is the first document.',
...     'This document is the second document.',
...     'Is this the first document?',]
vectorizer = CountVectorizer()
X1 = vectorizer.fit_transform(corpus1)
print(vectorizer.get_feature_names())
print(X1.toarray())  

### 计算每个词的TF-IDF

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
vectorizer=CountVectorizer()

In [11]:
X=vectorizer.fit_transform(corpus)

In [12]:
np.shape(X)

(60000, 213294)

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer

In [14]:
transformer=TfidfTransformer()

In [15]:
tfidf=transformer.fit_transform(X)

In [16]:
tfidf.toarray

<bound method _cs_matrix.toarray of <60000x213294 sparse matrix of type '<class 'numpy.float64'>'
	with 5485430 stored elements in Compressed Sparse Row format>>

# 数据集拆分

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
Xtrain, Xtest, ytrain, ytest = train_test_split(tfidf, y, test_size=0.2, random_state=42)

# KNN分类器--模型训练

In [19]:
from sklearn.neighbors import KNeighborsClassifier

In [20]:
knn = KNeighborsClassifier()
knn.fit(Xtrain,ytrain)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

# 预测

In [22]:
prediction = knn.predict(Xtest)
score = knn.score(Xtest,ytest)
print(score)

0.88575
